In [ ]:
# import dependencies

import cv2
import sys
import os
import numpy as np
from random import randint

# Import everything needed to edit video clips
from moviepy.editor import *

In [ ]:
'''
This notebook, takes an input video file and runs each Object tracking algorithm with a specific object from the video.
The videos are then written into a particular folder and stack_videos function then stack up the 8 clips into a single video file
for visualization purpose.
'''

def single_object_tracking_main(input_path):
    # Set up tracker. Instead of CSRT, you can also use
    tracker_types = ['BOOSTING', 'MIL', 'KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    # Read video
    video = cv2.VideoCapture(input_path)
    # Exit if video not opened.
    if not video.isOpened():
        print('could not open the video file')
        sys.exit()
    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print('cannot read the video file')
        sys.exit()

    video.release()
    
    # select a bounding box using selectROI function from opencv
    # # Define an initial bounding box
    # bbox = (287, 23, 86, 320)

    # Uncomment the line below to select a different bounding box
    bbox1 = cv2.selectROI(frame, False)
    
    output_file_list = list()
    # setracker_type defines the OpenCV object tracker implementations
    for tracker_type in tracker_types:
        if tracker_type == 'BOOSTING':
            tracker = cv2.TrackerBoosting_create()
        elif tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        elif tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        elif tracker_type == 'TLD':
            tracker = cv2.TrackerTLD_create()
        elif tracker_type == 'MEDIANFLOW':
            tracker = cv2.TrackerMedianFlow_create()
        elif tracker_type == 'GOTURN':
            tracker = cv2.TrackerGOTURN_create()
        elif tracker_type == 'MOSSE':
            tracker = cv2.TrackerMOSSE_create()
        elif tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()
        print(tracker_type)
        frames_list = list()
        video = cv2.VideoCapture(input_path)
        while True:
            # Read a new frame
            ok, frame = video.read()

            if not ok:
                break
            # get the tickCount from opencv
            timer = cv2.getTickCount()
            # Initialize tracker with first frame and bounding box
            ok = tracker.init(frame, bbox1)
            # Update tracker
            ok, bbox = tracker.update(frame)
            # get frames per secind to be shown in the window frame
            fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
            # Draw bounding box
            if ok:
                # Tracking Success
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(frame, p1, p2, ((randint(64, 255), randint(64, 255), randint(64, 255))), 2, 1)
            else:
                # Tracking failure
                cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255),
                            2)
            # Display tracker type and FPS on frame
            cv2.putText(frame, tracker_type, (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 3)
            cv2.putText(frame, "FPS : " + str(int(fps)), (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)

            frames_list.append(frame)

        video.release()
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        each_frame = frames_list[0]

        output_folder = 'OutputFolder'
        if not os.path.exists(output_folder):
            os.makedirs(output_folder, 0o755)
        output_file_path = os.path.join(output_folder, tracker_type+".mp4")

        new_video = cv2.VideoWriter(output_file_path, fourcc, 24, (each_frame.shape[1], each_frame.shape[0]))
        # write each frame from the list to the new_video object
        for each_frame in frames_list:
            new_video.write(each_frame)
        
        new_video.release()
        output_file_list.append(output_file_path)
    return output_file_list
    

In [ ]:
def stack_videos(input_path):
    # the single_object_tracking_main takes the input_path and returns the list of output file paths for each of the algos
    output_list = single_object_tracking_main(input_path)
    list_clips = list()
    # we will then append the videoclips toa list
    for each_path in output_list:
        list_clips.append(VideoFileClip(each_path))
        
    print(len(list_clips))
    # convrt the video list into  an array
    data = np.array(list_clips)
    # define/reshape the array to a 2D array eith 4 columns
    clips = data.reshape( (2,4) )
    
    # stacking clips
    final = clips_array(clips)

    # showing final clip
    final.ipython_display(width=1920)

In [ ]:
input_path = '../InputFiles/run.mp4'
stack_videos(input_path)

In [ ]:
a = [1,2,3,4,5,6,7,8]
d = np.array(a)
d.reshape((2,4))